## Cat Classifier
Determines whether an image contains a cat<br>
Dataset taken from Andrew Ng's Coursera course

In [5]:
import cupy as np # Use Cuda
from h5py import File
from importlib import reload
from PIL import Image
from sandbox import model, layers, activations, costs, utils

In [19]:
# Load data
train_dataset = File('dataset\\train_catvnoncat.h5', 'r')
train_x = np.array(train_dataset['train_set_x'][:]) # Train set features
train_y = np.array(train_dataset['train_set_y'][:]) # Train set labels

test_dataset = File('dataset\\test_catvnoncat.h5', 'r')
test_x = np.array(test_dataset['test_set_x'][:]) # Test set features
test_y = np.array(test_dataset['test_set_y'][:]) # Test set labels

train_y = train_y.reshape((train_y.shape[0], 1))
test_y = test_y.reshape((test_y.shape[0], 1))

# Flatten and normalize
train_x = train_x.reshape(train_x.shape[0], -1) / 255
test_x = test_x.reshape(test_x.shape[0], -1) / 255

In [16]:
reload(model)
np.random.seed(1)

cat = model.Model(cuda=True)
cat.add(layers.Dense(units=20, activation=activations.ReLU()))
cat.add(layers.Dense(units=7, activation=activations.ReLU()))
cat.add(layers.Dense(units=5, activation=activations.ReLU()))
cat.add(layers.Dense(units=1, activation=activations.Sigmoid()))

cat.configure(learning_rate=0.0075, epochs=700, cost_type=costs.BinaryCrossentropy())
cat.train(train_x, train_y, verbose=True)
cat.save() # Save model parameters

Cost on epoch 0: 0.66251
Cost on epoch 100: 0.50727
Cost on epoch 200: 0.48794
Cost on epoch 300: 0.26004
Cost on epoch 400: 0.18716
Cost on epoch 500: 0.17301
Cost on epoch 600: 0.07039
Cost on epoch 700: 0.05136


In [20]:
# Assess model accuracy
pred_train = cat.predict(train_x, prediction_type=utils.Predictions.binary_classification) # Get model accuracy on training data
print('Training Accuracy: '  + str(np.round(np.sum((pred_train == train_y)/train_x.shape[0]), decimals=5)))
pred_test = cat.predict(test_x, prediction_type=utils.Predictions.binary_classification) # Get model accuracy on testing data
print('Testing Accuracy: '  + str(np.round(np.sum((pred_test == test_y)/test_x.shape[0]), decimals=5)))

# Use model on custom image
my_image = 'dataset\\cat.jpg' 
num_px = 64

image = np.array(Image.open(my_image).resize((num_px, num_px))) / 255 # Resize and normalize image, cast to NumPy array
image = image.reshape((1, num_px * num_px * 3)) # Flatten image array
my_predicted_image = cat.predict(image, prediction_type=utils.Predictions.binary_classification) # Predict custom image
print(f"Image guess: {my_predicted_image.item()}")

Training Accuracy: 0.99043
Testing Accuracy: 0.72
Image guess: 1
